# 출력층 설계 (Output layer)


**회귀 vs 분류 출력층 비교표**


| 항목           | **회귀 (Regression)**            | **이진 분류 (Binary Classification)**        | **다중 분류 (Multi-class Classification)** |
| ------------ | ------------------------------ | ---------------------------------------- | -------------------------------------- |
| **출력층 뉴런 수** | 1개                             | 1개                                       | 클래스 수만큼 (예: 3개 클래스 → 3개 뉴런)            |
| **활성화 함수**   | 없음 (`Identity` = 항등함수)         | `Sigmoid`                                | 없음 (출력은 로짓값, softmax는 loss 내부 처리)      |
| **손실 함수**    | `MSELoss`, `L1Loss` 등          | `BCELoss`, `BCEWithLogitsLoss`           | `CrossEntropyLoss` (Softmax 포함)        |
| **정답 레이블**   | 실수 (float32), shape = `(n, 1)` | 0 또는 1 (float or long), shape = `(n, 1)` | 정수 (long), shape = `(n,)`              |
| **예측 방식**    | 그대로 출력 사용 (`ŷ`)                | `ŷ >= 0.5` → 1, else 0                   | `argmax(output, dim=1)`                |


<br/>


> 회귀는 **출력값에 제한이 없으므로** 아무 활성화도 적용하지 않음
>
> 이진 분류는 **확률**을 출력해야 하므로 sigmoid를 씌움. `BCEWithLogitsLoss` 사용하는 경우에는 **출력층에서는 sigmoid를 쓰지 않음**
>
> 다중 분류는 `CrossEntropyLoss`가 내부적으로 `Softmax` + `Log`를 처리하므로 **출력층에서는 softmax를 쓰지 않음**


## 회귀 출력층
항등함수란? $f(x) = x$와 같이 입력이 곧 출력인 함수를 가리킨다.
torch모델에서는 출력층 다음에 아무 활성화 함수를 사용하지 않는다.

In [ ]:
import filelock
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from torch.nn.functional import threshold

In [ ]:
# 데이터 생성
# X (100, 2)
# y (100, 1)

X = torch.randn(100, 2)
W = torch.tensor([[3., 2.]]) # (1, 2)
b = torch.tensor([5.])
noise = torch.randn(100, 1) * 2


y = X @ W.T + b + noise
# ==> (100, 2) @ (1, 2) 라서 내적이 안되므로 전치해준다


print(y)
print(y.shape)
# => torch.Size([100, 1])




In [ ]:
# 모델 생성
class RegressionNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.hiiden = nn.Linear(input_dim, 10) # 입력 2 -> 출력 10
        # ==> input_dim : 입력 특성 수에 따라서 바뀌게 됨
        self.relu = nn.ReLU() # ==> 활성화 함수
        self.output = nn.Linear(10,1) # 입력 10 -> 출력 1
        # ==> 회귀 이므로 1개 고정

    def forward(self, x):
        x = self.hiiden(x) # ==> 은닉층
        x = self.relu(x)   # ==>
        x = self.output(x) # ==> 출력층
        return x

model = RegressionNet(input_dim=X.size(1))
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)



In [ ]:
# 학습
for epoch in range(1000):
    model.train() # 학습모드
    optimizer.zero_grad() # ==> optimizer 초기화
    pred = model(X)
    loss = criterion(pred, y)
    loss.backward() # 기울기 계산
    optimizer.step() # 가중치 갱신

    if (epoch + 1) % 100 == 0 :
        print(f'Epoch : {epoch + 1} / Loss : {loss.item()}')


In [ ]:
# 예측 / 시각화

# torch.no_grad() 블럭 : 자동미분 연산 안하는 블럭(loss.backward() 가용 불가)
# model.eval() : 모델의 평가모드 활성화 (Dropout 처리)
# 드랍아웃 : 학습 중에 일부 뉴런을 랜덤으로 꺼서 모델이 특정 뉴런에만 의존하지 못하게 만드는 과적합 방지 기법
with torch.no_grad():
    model.eval()
    pred = model(X)

plt.scatter(y, pred)
plt.plot(
    [y.min(), y.max()],
    [y.min(), y.max()],
    'r--', label='True') # ==> 정답 선
plt.xlabel('Actual') # ==> 실제 값
plt.ylabel('Prediction') # ==> 예측값?
plt.grid()
plt.legend()


plt.show()

In [ ]:
# 은닉층 가중치/절편 shape : (10, 2) (10, )
# 출력층 가중치/절편 shape : (1, 10) (1, )

for name, param in model.named_parameters():
    print(f'{name} : {param.shape}')


## 캘리포니아 집값 예측

In [2]:
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split

X, y = fetch_california_housing(as_frame=True, return_X_y=True)
print(X.shape, y.shape)

# 데이터 전처리
X_scaler = StandardScaler()
y_scaler = StandardScaler() # 딥러닝은 학습안정성을 이유로 라벨 스케일링도 진행

X = X_scaler.fit_transform(X)
y = y_scaler.fit_transform(y.values.reshape(-1, 1))

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42) # ==> 회귀이므로 st뭐시기 없음

# - tensor 변환
# X_train = torch.tensor(X_train)
# X_test = torch.tensor(X_test)
# y_train = torch.tensor(y_train)
# y_test = torch.tensor(y_test)
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [ ]:
# 모델 생성

class CaliforniaHousingNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.hiiden1 = nn.Linear(input_dim, 32)
        self.relu1 = nn.ReLU()
        self.hidden2 = nn.Linear(32, 16)
        self.relu2 = nn.ReLU()
        self.output = nn.Linear(16, 1)

    def forward(self, x):
        x = self.hiiden1(x)
        x = self.relu1(x)
        x = self.hidden2(x)
        x = self.relu2(x)
        return x


In [ ]:
# 모델 생성

class CaliforniaHousingNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()

        # 모듈 겍체를 순서대로 묶고 실행
        self.net = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1)
        )

    def forward(self, x):
        return self.net(x)

model = CaliforniaHousingNet(input_dim=X.shape[1])
# ==> X 가 데이터프레임이므로 X.shape(1)이 아니라 X.shape[1]
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# 모델 학습

# 학습모드 활성화
model.train()

# epoch수 만큼 반복
for epoch in range(1000) :
    # 최적화함수 초기화
    optimizer.zero_grad()

    # 모델 예측
    pred = model(X_train)

    # 손실 계산
    loss = criterion(pred, y_train)

    # 역전파(기울기 계싼)
    loss.backward()

    # 가중치/절편 업데이트
    optimizer.step()

    # 100번마다 손실 로깅
    if (epoch + 1) % 100 == 0 :
        print(f'Epoch : {epoch + 1} / Loss : {loss.item()}')

# => RuntimeError: mat1 and mat2 must have the same dtype,
#                  but got Double and Float
print(X_train.dtype) # ==> torch.float64 이 들엉라서

In [ ]:
# 모델 학습

# 학습모드 활성화
model.train()

# epoch수 만큼 반복
for epoch in range(1000) :
    # 최적화함수 초기화
    optimizer.zero_grad()

    # 모델 예측
    pred = model(X_train)

    # 손실 계산
    loss = criterion(pred, y_train)

    # 역전파(기울기 계싼)
    loss.backward()

    # 가중치/절편 업데이트
    optimizer.step()

    # 100번마다 손실 로깅
    if (epoch + 1) % 100 == 0 :
        print(f'Epoch : {epoch + 1} / Loss : {loss.item()}')


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score, root_mean_squared_error, mean_absolute_error, accuracy_score

# 평가/시각화
model.eval()

with torch.no_grad() :
    pred = model(X_test)
# pred

# 라벨 스케일링값을 원복
y_test_inv = y_scaler.inverse_transform(y_test)
pred_inv = y_scaler.inverse_transform(pred)

# 평가
print(f'R^2 : {r2_score(y_test_inv, pred_inv)}')
print(f'MSE : {mean_squared_error(y_test_inv, pred_inv)}')
print(f'MAE : {mean_absolute_error(y_test_inv, pred_inv)}')
print(f'RMSE : {root_mean_squared_error(y_test_inv, pred_inv)}')

In [ ]:
# 시각화
# - 산점도 : x축 실제값, y축 실제값
# - 선그래프(기준선) : (0, 0) -> (최대실제값, 최대실제값)

plt.scatter(y_test_inv, pred_inv)
plt.plot(
    [0, y_test_inv.max()], [0, y_test_inv.max()],
    'r--', label='True'
)
plt.xlabel('Actual')
plt.ylabel('Prediction')
plt.grid()
plt.legend()

plt.show()

In [ ]:
# 모델 생성

class CaliforniaHousingNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()

        # 모듈 겍체를 순서대로 묶고 실행
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )

    def forward(self, x):
        return self.net(x)

model = CaliforniaHousingNet(input_dim=X.shape[1])
# ==> X 가 데이터프레임이므로 X.shape(1)이 아니라 X.shape[1]
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

### 이진분류
- Sigmoid 활성화 함수 사용
- 은닉층/출력층을 거쳐온 결과값(z)를 확률값(p)으로 변환
- 설정한 임계치 이상이면 양성으로 예측

In [ ]:
torch.manual_seed(42)
# ==> 결과값 고정

# 평균 0, 표준편차 10인 정규분포 샘플링 10개
z = torch.randn(10) * 10
# print(f'z : {z}')

p = F.sigmoid(z)
# print(f'p : {p}')

# 더 많은 양성클래스를 확보하고 싶다면(재현율), 임계치를 낮추면 된다
# 양성클래스의 정밀도를 높이고 싶다면, 임계치를 높이면 된다.
threshold = 0.5 # ==> 임계치
pred = (p >= threshold).int() # ==> 값계산 + 형변환
# pred
# => tensor([1, 1, 1, 1, 0, 0, 1, 0, 1, 1], dtype=torch.int32)

pd.DataFrame({
    'z' : z,
    'p' : p,
    'pred' : pred,
})



### 다중분류
- Softmax 활성화함수 사용
- 각 클래스별 계산값을 입력으로 받아, 각 클래스별 확률값으로 변환 (모든 클래스의 확률값 합 1)
- 벡터를 입력받아 벡터로 변환

In [ ]:
# 데이터샘플이 한건인 경우
z = torch.tensor([2., 1.5, 4, 0.7])
print(z)
output = F.softmax(z, dim=0)
print(output)



In [ ]:
# 데이터샘플이 여러 건인 경우
z = torch.tensor([[2., 1.5, 4, 0.7],
                  [3., 1, 4.7, 5]])
print(z)
output = F.softmax(z, dim=1)
print(output)
# => tensor([[2.0000, 1.5000, 4.0000, 0.7000],
#            [3.0000, 1.0000, 4.7000, 5.0000]])
# => tensor([[0.1079, 0.0654, 0.7973, 0.0294],
#            [0.0714, 0.0097, 0.3910, 0.5279]])

pred = output.argmax(dim=1)
print(pred)
# => tensor([2, 3])


# softmax 결과의 합은 항상 1
print(output.sum(dim=1))
# => tensor([1.0000, 1.0000])


In [ ]:
# (참고) 다중클래스 예측에 sigmoid를 사용하는 경우
z = torch.tensor([[2., 1.5, 4, 0.7],
                  [3., 1, 4.7, 5]])
print(z)

# 각 클래스별로 양성일 확률을 반환 (모두의 합이 1이 아니다)
p = F.sigmoid(z)
print(p)

# 예측
pred = p.argmax(dim=1)
print(pred)

### 출력층과 손실함수 연계

**이진분류**
- 출력층 sigmoid + 손실함수 BCLoss
- 출력층 x + 손실함수 BCEWithLogitsLoss


**다중분류**
- 출력층 x + 손실함수 CrossEntropyLoss

#### 이진분류 : 출력층 sigmoid + 손실함수 BECLoss

In [ ]:
from sklearn.datasets import  make_classification

# 데이터 생성
X, y = make_classification(
    n_samples=100, # 데이터 수
    n_features=10, # 특성 수
    n_informative=5, # 중요한 속성 수
    n_classes=2, # 예측클래스 (이진분류)
    random_state=42
)

print(X.shape, y.shape)
# => (100, 10) (100,)

# 데이터 준비 (tensor 타입)
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32).reshape(-1, 1) # 2차원 변경

print(X.shape, y.shape)
# => torch.Size([100, 10]) torch.Size([100, 1])

In [ ]:
# 모델 생성

class BinaryClassificationNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)

# ==> model, criterion, optimizer 준비
model = BinaryClassificationNet(input_dim=X.shape[1]) # 입력데이터 특성수
# => 10
criterion = nn.BCELoss() # 출력층 sigmoid가 반환한 확률값을 가지고 손실 계산
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# 모델 학습
epochs = 100
model.train()

for epoch in range(epochs) :
    optimizer.zero_grad()
    pred = model(X)
    # ==> 예측해봐
    loss = criterion(pred, y) # 100개 샘플 오차의 평균값(스칼라)
    # ==> 보통 평균이나 합계 단일값으로 함
    # ==> 손실 계산해봐
    loss.backward()
    # ==> 손실 줄이게 기울기 계산해봐
    optimizer.step()
    # ==> 그걸고 가중치 업데이트 해봐?

    if (epoch + 1) % 10 == 0 :
        print(f'Epoch : {epoch + 1} / Loss : {loss.item()}')


#### 출력층 x + 손실함수 BCEWithLogitLoss

In [ ]:
# 모델 생성

class BinaryClassificationNet2(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 1),
        )

    def forward(self, x):
        return self.net(x) # sigmoid 활성화함수를 사용하지 않으므로,
                           # 확률값이 아닌 선형방정식 값이 반환

# ==> model, criterion, optimizer 준비
model = BinaryClassificationNet2(input_dim=X.shape[1]) # 입력데이터 특성수
criterion = nn.BCEWithLogitsLoss() # 내부적으로 sigmoid 활성화함수 처리
# ==> 주의!! : sigmoid를 두번 돌리지 않게 조심하기
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [ ]:
# 모델 학습
epochs = 100
model.train()

for epoch in range(epochs) :
    optimizer.zero_grad()
    logits = model(X)
    loss = criterion(logits, y) # 100개 샘플 오차의 평균값(스칼라)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0 :
        print(f'Epoch : {epoch + 1} / Loss : {loss.item()}')


유방암 예측

In [1]:
from sklearn.datasets import  load_breast_cancer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



X, y = load_breast_cancer(return_X_y=True)
print(X.shape, y.shape)
# => (569, 30) (569,)

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y)

# 데이터 전처리
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
# ==> 이진분류이므로 y 전처리 없음
print(X_train.dtype) # float64


# Tensor 데이터 준비
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(-1)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(-1)
# ==> unsqueeze로 차원 늘리기

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
# => torch.Size([455, 30]) torch.Size([455])
# => torch.Size([114, 30]) torch.Size([114])
# ===> unsqueeze 후
# => torch.Size([455, 30]) torch.Size([455, 1])
# => torch.Size([114, 30]) torch.Size([114, 1])

(569, 30) (569,)


In [ ]:
# 모델 작성
class BreastCancerNet(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 32),
            nn.ReLU(),
            nn.Linear(32, 16),
            nn.ReLU(),
            nn.Linear(16, 1),
        )

    def forward(self, x):
        return self.net(x)



# 모델 / 손실함수 / 최적화함수 선언
model = BreastCancerNet(input_dim=X_train.size(1))
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [ ]:
# 모델 학습
model.train()

for epoch in range(100) :
    optimizer.zero_grad()
    pred = model(X_train) # 로짓
    # ==> 변수명은 pred지만 sigmoid가 없으므로 실제 들어오는건 logits값 이다
    loss = criterion(pred, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0 :
        print(f'Epoch : {epoch + 1} / Loss : {loss.item()}')


In [ ]:
from sklearn.metrics import accuracy_score

# 모델 평가
model.eval()
with torch.no_grad() : # ==> 기울기 계산할 필요 없다
    logits = model(X_test)
    # print(logits) # ==> 로짓값이 나온다
    p = F.sigmoid(logits)
    # print(p) # ==> 0에서 1 사이 값 나온다
    pred = (p >= 0.5).int()
    # print(pred) # ==> 이진분류 (0 또는 1)값으로 나온다
    print('정확도 : ', accuracy_score(y_test, pred))


#### 출력층 x + 손실함수 CrossEntropyLoss

In [ ]:
# 데이터 생성
X = torch.randn(4, 5) # 정규분포 4개의 데이터샘플, 5개의 특성
# ==> 총 20개
y = torch.tensor([0, 2, 1, 0]) # 라벨 자료형 long
n_classes = len(y.unique())
# n_classes
# => 3

# 모델 생성
class MultiClassficationNet(nn.Module):
    def __init__(self, input_dim, n_classes):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 32), # 은닉층
            nn.ReLU(), # 은닉층 활성화함수
            # nn.Linear(32, 3) # 출력층
            # ==> 다중분류 정답이 3개이므로 노드 3개
            nn.Linear(32, n_classes) # 출력층
            # ==> 정답 개수가 바뀔 수 있으므로 변수화
        )

    def forward(self, x):
        return self.net(x)

model = MultiClassficationNet(input_dim=X.size(1), n_classes=n_classes)
criterion = nn.CrossEntropyLoss() # softmax 내장
optimizer = optim.Adam(model.parameters(), lr=0.01)


In [ ]:
# 모델 학습
model.train()

for epoch in range(100) :
    optimizer.zero_grad()
    logits = model(X) # 클래스별 로짓
    # print(logits.shape)
    loss = criterion(logits, y)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0 :
        print(f'Epoch : {epoch + 1} / Loss : {loss.item()}')


##### 붓꽃데이터 예측

In [ ]:
from sklearn.datasets import load_iris

X, y = load_iris(return_X_y=True)
print(X.shape, y.shape)
# => (150, 4) (150,)


# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# 스케일링
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

print(X_train.dtype)
# => float64

# Tensor 변환
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long)

print(y_train.dtype)
# => torch.int64

print(X_train.shape, y_train.shape)
print(X_test.shape, y_test.shape)
# => torch.Size([120, 4]) torch.Size([120])
# => torch.Size([30, 4]) torch.Size([30])

In [ ]:
# 모델 생성
class IrisNet(nn.Module):
    def __init__(self, input_dim, output_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, output_dim)
        )

    def forward(self, x):
        return self.net(x)

model = IrisNet(input_dim=X_train.size(1), output_dim=3)
# ==> y_train에서 unique값 꺼내서 output_dim에 넣어도 상관없음

criterion = nn.CrossEntropyLoss() # softmax 내장
optimizer = optim.Adam(model.parameters(), lr=0.01)

# 모델 학습
model.train()

for epoch in range(100) :
    optimizer.zero_grad()
    logits = model(X_train)
    # print(logits) # ==> 모르겠으면 찍어봐라

    loss = criterion(logits, y_train)
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0 :
        print(f'Epoch : {epoch + 1} / Loss : {loss.item()}')

# 모델 평가
model.eval()
with torch.no_grad() :
    logits = model(X_test)
    print('logits : ', logits[:2])
    p = F.softmax(logits, dim=1)
    print('p : ', p[:2])
    pred = p.argmax(dim=1) # 열 고정 : 행간 가장 큰 확률값의 인덱스
    # ==> dim=1 행별로 놓고 봤을 때 가장 큰 확률값의 인덱스
    print('pred : ', pred[:2])
    print('y_true : ', y_test[:2])

    print('정확도 : ', accuracy_score(y_test, pred))
# => logits :  tensor([[  7.0337,  -2.9967, -11.4675],
#                      [ -5.4418,   1.2109,   2.4510]])
# =>      p :  tensor([[9.9996e-01, 4.4038e-05, 9.2256e-09],
#                      [2.8955e-04, 2.2435e-01, 7.7536e-01]])
# =>   pred :  tensor([0, 2])
# => y_true :  tensor([0, 2])
# =>  정확도 :  0.9666666666666667